SubQueries,CTE's,CASES

In [0]:
CREATE TABLE Customer (
    CustomerID INT PRIMARY KEY,
    FirstName VARCHAR(50),
    LastName VARCHAR(50),
    Email VARCHAR(100),
    City VARCHAR(50)
);

INSERT INTO Customer (CustomerID, FirstName, LastName, Email, City) VALUES
(1, 'John', 'Doe', 'john.doe@example.com', 'New York'),
(2, 'Jane', 'Smith', 'jane.smith@example.com', 'Los Angeles'),
(3, 'Robert', 'Brown', 'robert.brown@example.com', 'New York'),
(4, 'Emily', 'Davis', 'emily.davis@example.com', 'Houston'),
(5, 'Michael', 'Wilson', 'michael.wilson@example.com', 'Chicago');


In [0]:
select * from Order_Details

In [0]:
CREATE TABLE Orders (
    OrderID INT PRIMARY KEY,
    CustomerID INT,
    OrderDate DATE,
    TotalAmount DECIMAL(10, 2),
    FOREIGN KEY (CustomerID) REFERENCES Customer(CustomerID)
);

INSERT INTO Orders (OrderID, CustomerID, OrderDate, TotalAmount) VALUES
(101, 1, '2023-07-01', 250.50),
(102, 2, '2023-07-03', 180.00),
(103, 1, '2023-07-04', 75.99),
(104, 3, '2023-07-05', 320.00),
(105, 4, '2023-07-06', 150.00),
(106, 1, '2023-07-07', 90.00),
(107, 3, '2023-07-08', 420.00);


In [0]:
CREATE TABLE Order_Details (
    OrderDetailID INT PRIMARY KEY,
    OrderID INT,
    ProductName VARCHAR(100),
    Quantity INT,
    Price DECIMAL(10, 2),
    FOREIGN KEY (OrderID) REFERENCES Orders(OrderID)
);

INSERT INTO Order_Details (OrderDetailID, OrderID, ProductName, Quantity, Price) VALUES
(1, 101, 'Laptop', 1, 250.50),
(2, 102, 'Mouse', 2, 30.00),
(3, 102, 'Keyboard', 1, 120.00),
(4, 103, 'USB Cable', 3, 25.33),
(5, 104, 'Monitor', 2, 160.00),
(6, 105, 'Webcam', 1, 150.00),
(7, 106, 'Notebook', 2, 45.00),
(8, 107, 'Tablet', 1, 420.00);


List customers who have placed at least one order with an amount greater than ₹200.

In [0]:
select distinct CustomerID,FirstName
from customer 
where
CustomerID IN(select CustomerID from orders where TotalAmount > 200)


Show all orders placed by customers who live in New York.

In [0]:
select  *
from orders 
where CustomerID IN (select CustomerID from customer where City = 'New York')

In [0]:
--select *from customer ;
select *from orders;
--select *from Order_Details;

Get the customer(s) who placed the most expensive order.

In [0]:
select 
concat(FirstName,' ',lastName) as Name,od.ProductName
 from customer c join orders o on c.CustomerID = o.CustomerID join order_details od on o.OrderID = od.OrderID where od.Price = (select max(Price) from Order_Details)

For each order, display the order ID and indicate if its amount is above the customer’s average order amount.

In [0]:
SELECT 
    o.OrderID,
    o.CustomerID,
    o.TotalAmount,
    CASE 
        WHEN o.TotalAmount > (
            SELECT AVG(o2.TotalAmount)
            FROM Orders o2
            WHERE o2.CustomerID = o.CustomerID
        ) THEN 'Yes'
        ELSE 'No'
    END AS AboveAverage
FROM 
    Orders o;


CTEs – Cleaner and Modular Queries


Use a CTE to calculate total order amount per customer and then select only those who spent more than ₹200.

In [0]:
With total_order AS (
select 
    CustomerID,
    sum(TotalAmount) as total
 from orders  group by CustomerID
)
select ot.CustomerID,ot.total
from total_order ot join customer c on ot.CustomerID = c.CustomerID where ot.total > 200

Generate numbers from 1 to 10 using a recursive CTE.



In [0]:
WITH RECURSIVE Numbers AS (
    SELECT 1 AS n
    UNION ALL
    SELECT n + 1
    FROM Numbers
    WHERE n < 10
)
SELECT n
FROM Numbers;
---Recursive CTEs not supportedin databricks

First CTE: total orders per customer.
Second CTE: customers with more than 1 orders.
Final output: names of such customers.

In [0]:
WITH Total_orders AS (
select 
    concat(c.FirstName,' ',c.lastName) as Name, count(*) as COUNT from orders o join workspace.default.customer c ON o.CustomerID = c.CustomerID GROUP BY c.CustomerID,concat(c.FirstName,' ',c.lastName)
),

 ORDER_MORE_THAN_1 AS (
 SELECT Name FROM Total_orders WHERE  COUNT> 1
)

SELECT * FROM ORDER_MORE_THAN_1

CASE Statements – Logic Inside SQL

Use CASE to show whether an order is labeled as 'High', 'Medium', or 'Low' based on TotalAmount

In [0]:
select 
OrderID, 
 CASE
 when TotalAmount > 250 then 'High'
 when TotalAmount > 150 then 'Medium'
 else 'Low'
END AS Amount
 from orders

Display each customer’s name and tag them as 'Metro' or 'Non-Metro' depending on their city.

In [0]:
select city,
case 
when city = 'New York' then 'Metro'
when city = 'Los Angeles' then 'Metro'
when city = 'Chicago' then 'Metro'
when city = 'Houston' then 'Non-Metro'
when city = 'Phoenix' then 'Non-Metro'
END AS CITYSTATUS
from customer

Use CASE to return:

'Frequent Buyer' if a customer has more than 1 orders

'One-Time Buyer' if exactly 1

'Inactive' if none

In [0]:
SELECT 
    c.CustomerID,
    CONCAT(c.FirstName, ' ', c.LastName) AS CustomerName,
    COUNT(o.OrderID) AS OrderCount,
    CASE 
        WHEN COUNT(o.OrderID) > 1 THEN 'Frequent Buyer'
        WHEN COUNT(o.OrderID) = 1 THEN 'One-Time Buyer'
        WHEN COUNT(o.OrderID) = 0 THEN 'Inactive'
        
    END AS BuyerType
FROM 
    Customer c
LEFT JOIN 
    Orders o ON c.CustomerID = o.CustomerID
GROUP BY 
    c.CustomerID, c.FirstName, c.LastName;
